# EDA - Log Files

<p>This notebook is to analyze the Data files to understand how to load the data, Data Quality, Define Data Types & Define Constraints</p>

In [1]:
import pandas as pd
import os
from itertools import zip_longest
import glob
import numpy as np
import shutil

In [2]:
def FUNC_GetAllSubFiles(param_dir):
    """
    Iterates through all child Directories and returns a list of all .json Files excluding checkpoint files create by notebooks
    """
    LIST_AllDirs = []
    
    for dir_path, dir_names, file_names in os.walk(param_dir):
        # Excluding Checkpoint Files 
        file_names = list(filter(lambda filename : filename.find('checkpoint')==-1 and filename.find('.json')!=-1, file_names))
        
        if file_names != []:# Excluding empty Directories
            LIST_AllDirs.extend(list(zip_longest([dir_path],file_names,fillvalue=dir_path)))
        
    return tuple(map(lambda x : os.path.join(x[0],x[1]),LIST_AllDirs))

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [4]:
# A List of all JSON Files in this Directory
TUPLE_AllFiles = get_files('data/log_data/')

In [5]:
sorted(TUPLE_AllFiles)[:10]

['/home/workspace/data/log_data/2018/11/2018-11-01-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-02-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-03-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-04-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-05-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-06-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-07-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-08-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-09-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-10-events.json']

In [6]:
DF_AllLogs = pd.DataFrame(columns=['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId','SRC_FILE'])

In [7]:
for filePath in TUPLE_AllFiles:
    DF_Temp = pd.read_json(filePath, lines=True)
    DF_Temp["SRC_FILE"] = filePath.split('/')[-1]
    DF_AllLogs = DF_AllLogs.append(DF_Temp)

In [8]:
DF_AllLogs.sort_values('ts',inplace=True)

In [9]:
DF_AllLogs.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,SRC_FILE
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39,2018-11-01-events.json
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39,2018-11-01-events.json
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01-events.json
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01-events.json
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01-events.json


### Analyzing and Preparing Timestamp Data

In [10]:
DF_TimeStamp = DF_AllLogs[["ts"]].copy().rename(columns={'ts':'TIME_ID'})

In [11]:
DF_TimeStamp.head()

,TIME_ID
0,1541105830796
0,1541105830796
2,1541106106796
1,1541106106796
2,1541106106796


#### Checking Uniqueness

In [12]:
len(DF_TimeStamp.TIME_ID.unique())

8023

In [13]:
len(DF_TimeStamp.TIME_ID)

8071

Not Unique Values

In [14]:
# Parsing Unix Timesatmp into Readable Timestamp
DF_TimeStamp["TS_FORMATTED"] = pd.to_datetime(DF_TimeStamp.TIME_ID,unit='ms')

In [15]:
DF_TimeStamp.head()

,TIME_ID,TS_FORMATTED
0,1541105830796,2018-11-01 20:57:10.796
0,1541105830796,2018-11-01 20:57:10.796
2,1541106106796,2018-11-01 21:01:46.796
1,1541106106796,2018-11-01 21:01:46.796
2,1541106106796,2018-11-01 21:01:46.796


In [16]:
DF_TimeStamp["YEAR_COL"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).year
DF_TimeStamp["MONTH_COL"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).month
DF_TimeStamp["DAY_COL"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).day
DF_TimeStamp["HOUR_COL"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).hour
DF_TimeStamp["MINUTE_COL"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).minute
DF_TimeStamp["SECOND_COL"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).second
DF_TimeStamp["WEEK_DAY"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).weekday
DF_TimeStamp["DAY_NAME"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).day_name()

In [17]:
DF_TimeStamp.head()

,TIME_ID,TS_FORMATTED,YEAR_COL,MONTH_COL,DAY_COL,HOUR_COL,MINUTE_COL,SECOND_COL,WEEK_DAY,DAY_NAME
0,1541105830796,2018-11-01 20:57:10.796,2018,11,1,20,57,10,3,Thursday
0,1541105830796,2018-11-01 20:57:10.796,2018,11,1,20,57,10,3,Thursday
2,1541106106796,2018-11-01 21:01:46.796,2018,11,1,21,1,46,3,Thursday
1,1541106106796,2018-11-01 21:01:46.796,2018,11,1,21,1,46,3,Thursday
2,1541106106796,2018-11-01 21:01:46.796,2018,11,1,21,1,46,3,Thursday


In [18]:
def FUNC_ProcessTimestampData(param_df,param_col_name):
    """Recieves a Data Frame contains a single Columns of Unix time stamp and returns a DataFrame Contains all TIME_TBL Columns"""
    try :
        DF_TimeStamp = param_df[[param_col_name]].copy()
        DF_TimeStamp["TS_FORMATTED"] = pd.to_datetime(param_df[param_col_name],unit='ms')
        DF_TimeStamp["YEAR_COL"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).year
        DF_TimeStamp["MONTH_COL"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).month
        DF_TimeStamp["DAY_COL"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).day
        DF_TimeStamp["HOUR_COL"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).hour
        DF_TimeStamp["MINUTE_COL"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).minute
        DF_TimeStamp["SECOND_COL"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).second
        DF_TimeStamp["WEEK_DAY"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).weekday
        DF_TimeStamp["DAY_NAME"] = pd.DatetimeIndex(DF_TimeStamp.TS_FORMATTED).day_name()
        
        DF_TimeStamp.rename(columns={'ts':'TIME_ID'} , inplace=True)
        
        DF_TimeStamp.drop_duplicates('TIME_ID',inplace=True)
        
        return DF_TimeStamp
    except Exception as  e:
        print("ERROR: {}".format(e))

In [19]:
FUNC_ProcessTimestampData(DF_AllLogs[["ts"]].copy(),'ts').head()

,TIME_ID,TS_FORMATTED,YEAR_COL,MONTH_COL,DAY_COL,HOUR_COL,MINUTE_COL,SECOND_COL,WEEK_DAY,DAY_NAME
0,1541105830796,2018-11-01 20:57:10.796,2018,11,1,20,57,10,3,Thursday
2,1541106106796,2018-11-01 21:01:46.796,2018,11,1,21,1,46,3,Thursday
3,1541106132796,2018-11-01 21:02:12.796,2018,11,1,21,2,12,3,Thursday
4,1541106352796,2018-11-01 21:05:52.796,2018,11,1,21,5,52,3,Thursday
5,1541106496796,2018-11-01 21:08:16.796,2018,11,1,21,8,16,3,Thursday


In [20]:
len(FUNC_ProcessTimestampData(DF_AllLogs[["ts"]].copy(),'ts'))

8023

In [21]:
len(FUNC_ProcessTimestampData(DF_AllLogs[["ts"]].copy(),'ts').TIME_ID.unique())

8023

## Analyzing and Preparing User Data

In [22]:
DF_AllLogs[['userId','firstName','lastName','gender','level']].head()

,userId,firstName,lastName,gender,level
0,39,Walter,Frye,M,free
0,39,Walter,Frye,M,free
2,8,Kaylee,Summers,F,free
1,8,Kaylee,Summers,F,free
2,8,Kaylee,Summers,F,free


In [23]:
DF_AllLogs.gender.unique()

array(['M', 'F', None], dtype=object)

In [24]:
DF_AllLogs.level.unique()

array(['free', 'paid'], dtype=object)

Checking Missing Data

In [25]:
len(DF_AllLogs)

8071

In [26]:
DF_AllLogs[['userId','firstName','lastName','gender','level']].isnull().sum()

userId         0
firstName    286
lastName     286
gender       286
level          0
dtype: int64

In [27]:
DF_AllLogs.applymap(lambda x:x=="").any()

artist           False
auth             False
firstName        False
gender           False
itemInSession    False
lastName         False
length           False
level            False
location         False
method           False
page             False
registration     False
sessionId        False
song             False
status           False
ts               False
userAgent        False
userId            True
SRC_FILE         False
dtype: bool

In [28]:
len(DF_AllLogs[(DF_AllLogs.userId == "")])

286

Note, All Missing data is just due to User is not logged in, so we will neglect them while creating user Table

## Analyzing and Preparing Users Data

In [29]:
def FUNC_ProcessUserData(param_df):
    """A Function to Process User Table Data, recieves a Dataframe then selects, processes & returns User Data ready to be inserted in the Db"""
    try:
        
        # Filtering on Logged user only, Sorting the Data to preserve the latest Status while removing duplicates & Selecting the Required Columns
        DF_UserData = param_df[param_df.auth== 'Logged In'][['userId','firstName','lastName','gender','level','ts']].copy()
        DF_UserData.sort_values('ts',inplace=True)
        DF_UserData.drop(['ts'], axis=1, inplace=True)
        DF_UserData.userId = DF_UserData.userId.astype('int')
        
        DF_UserData.drop_duplicates('userId',inplace=True,keep='last')
        
        return DF_UserData
    except Exception as e:
        print("ERROR: {}".format(e))

In [30]:
len(DF_AllLogs[DF_AllLogs.auth== 'Logged In'].userId.astype('int').unique())

97

In [31]:
len(FUNC_ProcessUserData(DF_AllLogs).userId.unique())

97

In [32]:
FUNC_ProcessUserData(DF_AllLogs).sort_values('userId')

,userId,firstName,lastName,gender,level
39,2,Jizelle,Benjamin,F,free
170,3,Isaac,Valdez,M,free
8,4,Alivia,Terrell,F,free
387,5,Elijah,Davis,M,free
214,6,Cecilia,Owens,F,free
5,7,Adelyn,Jordan,F,free
168,8,Kaylee,Summers,F,free
337,9,Wyatt,Scott,M,free
56,10,Sylvie,Cruz,F,free
357,11,Christian,Porter,F,free


## Analyzing and Preparing Song Play Data

Implement the song_select query in sql_queries.py to find the song ID and artist ID based on the title, artist name, and duration of a song.

Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to songplay_data

In [33]:
DF_SongPlayCols = DF_AllLogs[['ts','userId','level','song','artist','sessionId','location','userAgent','auth']].copy()

In [34]:
DF_SongPlayCols.head()

,ts,userId,level,song,artist,sessionId,location,userAgent,auth
0,1541105830796,39,free,None,None,38,"San Francisco-Oakland-Hayward, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Logged In
0,1541105830796,39,free,None,None,38,"San Francisco-Oakland-Hayward, CA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",Logged In
2,1541106106796,8,free,You Gotta Be,Des'ree,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Logged In
1,1541106106796,8,free,None,None,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Logged In
2,1541106106796,8,free,You Gotta Be,Des'ree,139,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Logged In


In [35]:
DF_SongPlayCols.dtypes

ts           object
userId       object
level        object
song         object
artist       object
sessionId    object
location     object
userAgent    object
auth         object
dtype: object

In [36]:
len(DF_SongPlayCols)

8071

In [37]:
len(DF_SongPlayCols.drop_duplicates())

8052

In [38]:
DF_SongPlayCols.level.unique()

array(['free', 'paid'], dtype=object)

In [39]:
max(DF_SongPlayCols.location.str.len())

46.0

In [40]:
max(DF_SongPlayCols.userAgent.str.len())

139.0

In [41]:
DF_SongPlayCols.auth.unique()

array(['Logged In', 'Logged Out'], dtype=object)

In [42]:
len(DF_SongPlayCols[['ts','userId','level','song','artist','sessionId','location','userAgent','auth']].drop_duplicates())

8052

In [43]:
len(DF_AllLogs[['ts','sessionId']].drop_duplicates())

8037

In [44]:
DF_AllLogs[(DF_AllLogs.ts==1541106106796) & (DF_AllLogs.sessionId==139)]

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,SRC_FILE
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01-events.json
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01-events.json
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01-events.json
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01-events.json


In [45]:
DF_AllLogs[(DF_AllLogs.ts==1543582902796) & (DF_AllLogs.sessionId==1096)]

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,SRC_FILE
187,Big Tymers,Logged In,Chloe,F,7,Cuevas,252.05506,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,1096,Against The Wall,200,1543582902796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49,2018-11-30-events.json
188,None,Logged In,Chloe,F,8,Cuevas,NaN,paid,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540941e+12,1096,None,200,1543582902796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49,2018-11-30-events.json


In [46]:
DF_AllLogs[(DF_AllLogs.ts==1541106106796) & (DF_AllLogs.sessionId==139)]

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,SRC_FILE
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01-events.json
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01-events.json
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01-events.json
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8,2018-11-01-events.json


In [47]:
DF_SongPlayCols.song.str.len().max()

151.0

In [48]:
DF_SongPlayCols.artist.str.len().max()

89.0

In [49]:
len(DF_AllLogs[(DF_AllLogs.userId == "")].drop_duplicates())

286

In [50]:
def FUNC_ProcessSongPlayTempData(param_df):
    """A Function to Process Song Play Table Data, recieves a Dataframe then selects, processes & returns Song Play Data ready to be inserted in the Db"""
    try:
        DF_SongPlayData = param_df[param_df.page == 'NextSong'] #Filtering on 'NextSong' because this is where Song Data Available
        DF_SongPlayData = DF_SongPlayData[['ts','sessionId','artist','song','userId','level','location','userAgent','auth','length']].copy().drop_duplicates(['ts','sessionId'])
        DF_SongPlayData.userId = DF_SongPlayData.userId.apply(lambda x: -9999 if x in (np.nan,'') else x)
        
        return DF_SongPlayData
    except Exception as e:
        print("ERROR: {}".format(e))

In [51]:
DF_ = FUNC_ProcessSongPlayTempData(DF_AllLogs)

In [52]:
DF_.head()

,ts,sessionId,artist,song,userId,level,location,userAgent,auth,length
2,1541106106796,139,Des'ree,You Gotta Be,8,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Logged In,246.30812
4,1541106352796,139,Mr Oizo,Flat 55,8,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Logged In,144.03873
5,1541106496796,139,Tamba Trio,Quem Quiser Encontrar O Amor,8,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Logged In,177.18812
6,1541106673796,139,The Mars Volta,Eriatarka,8,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Logged In,380.42077
7,1541107053796,139,Infected Mushroom,Becoming Insane,8,free,"Phoenix-Mesa-Scottsdale, AZ","""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",Logged In,440.26730


In [53]:
len(DF_[DF_.userId == -9999])

0

In [54]:
len(FUNC_ProcessSongPlayTempData(DF_AllLogs))

6820

In [55]:
DF_AllLogs[DF_AllLogs.page != 'NextSong']['page'].unique()

array(['Home', 'Upgrade', 'Downgrade', 'Settings', 'Save Settings',
       'Login', 'Logout', 'Help', 'Error', 'About', 'Submit Upgrade',
       'Submit Downgrade'], dtype=object)

In [56]:
def FUNC_MoveProcessesFile(param_filepath):
    try:
        
        if os.path.isfile(param_filepath):
            #print(os.path.join('LOADED_DATA',*param_filepath.split('/')[1:]))
            LIST_ = param_filepath.split('/')
            STR_TrgtPath = os.path.join('/home/workspace/LOADED_DATA/log_data',*param_filepath.split('/')[-3:-2])
            STR_TrgtFile = os.path.join(STR_TrgtPath,param_filepath.split('/')[-1])
            print(STR_TrgtPath)
            print(STR_TrgtFile)
            
            if not os.path.isdir(STR_TrgtPath):
                os.makedirs(STR_TrgtPath)  
            
            #os.replace("path/to/current/file.foo", "path/to/new/destination/for/file.foo")
            shutil.copyfile(param_filepath, STR_TrgtFile)
            
        else:
            raise Exception("File Does Not Exist")
            
    except Exception as e:
        print("ERROR: {}".format(e))

In [57]:
FUNC_MoveProcessesFile('data/log_data/2018/11/2018-11-01-events.json')

/home/workspace/LOADED_DATA/log_data/2018
/home/workspace/LOADED_DATA/log_data/2018/2018-11-01-events.json


In [58]:
FUNC_MoveProcessesFile('data/song_data/A/A/A/TRAAAAW128F429D538.json')

/home/workspace/LOADED_DATA/log_data/A
/home/workspace/LOADED_DATA/log_data/A/TRAAAAW128F429D538.json
